# oneM2M - Basic Resources and Interactions
This notebook shows the basic interactions with a CSE using REST calls. Examples include:

- Create an &lt;AE> resource
- Create a &lt;Container> resource
- Create one or more &lt;ContentInstance> resources
- Retrieve the latest &lt;ContentInstance> resource
- Update the &lt;Container> resource
- Retrieve the &lt;Container> resource

## Intitialization
The section does import necessary modules and configurations.

In [ ]:
from init import *

## Create an &lt;AE> Resource
This example creates a new &lt;AE> resource in the CSE. &lt;AE>'s represent applications or services.

Creating this (or other) resource is done using a POST request and with providing a couple of mandatory attributes:

- **api** : Application Identifier. An Application Identifier uniquely identifies an M2M Application in a given context.
- **rr** : Request Reachability. This attribute indicates whether a resource can receive requests.

Note, that the request target is the &lt;CSEBase> resource. All create requests target a parent resource.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
}

data = '''
{
  "m2m:ae": {
    "rn": "myAE",
    "api": "AE",
    "rr": true
  }
}
'''

response = requests.post(url, headers=headers, data=data)    # CREATE request
printResponse(response)

The response introduces some new attributes:

- **pi** : This is the identifier of the parent resource.
- **et** : Expiration time/date after which the CSE will delete the resource.
- **aei** : An Application Entity Identifier uniquely identifies an AE globally.

**Note**: If you see an error "409" then don't worry. It just means that an &lt;AE> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Create a &lt;Container> Resource
In this section we add a &lt;Container> resource to the &lt;AE> resource. A &lt;Container> represents a data point that can hold a configurable number of dsta instances. A &lt;Container> may also hold a sub-containers.

If created with no further attributes, the CSE will assign defaults to the &lt;Container> resource.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
}

data = '''
{
  "m2m:cnt": {
    "rn":"myContainer"
  }
}
'''

response = requests.post(url + '/myAE', headers=headers, data=data)    # CREATE request
printResponse(response)

The following attributes are used with the &lt;Container> resource:

- **st** : The State Tag is incremented every time the resource is modified.
- **mni** : Maximum number of direct data instances in the &lt;Container> resource.
- **mbs** : Maximum size in bytes of data.
- **mia** : Maximum age of a direct data instances in the &lt;Container> resource.
- **cni** : Current number of direct data instances in the &lt;Container> resource.
- **cbs** : Current size in bytes of data.
- **ol** : Resource identifier of a virtual resource that points to the oldest data instance of the &lt;Container> resource.
- **la** : Resource identifier of a virtual resource that points to the latest data instance of the &lt;Container> resource.

**Note**: If you see an error "409" then don't worry. It just means that an &lt;Container> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Add a &lt;ContentInstance> to the &lt;Container>

Now, we add an actual value to the *myContainer* &lt;Container>. 

These attributes are part of the request:

- **cnf** : This specifies the content format. It specifies the media type as well as an encoding type.
- **con** : This is the actual content (ie. the value) that will be stored in the &lt;Container resource.

&lt;ContentInstance>'s can only be added and read, but not updated or deleted.

**Note**: You can execute the following code as often as you like in order to create more &lt;ContentInstance> resources.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
}

data = '''
{
  "m2m:cin": {
    "cnf": "text/plain:0",
    "con": "Hello, World!"
  }
}
'''

response = requests.post(url + '/myAE/myContainer', headers=headers, data=data)    # CREATE request
printResponse(response)

A new attribute:

- **cs** : This attribute contains the size of the content of the **con** attribute.

## Retrieve the latest &lt;ContentInstance> resource
This request will retrieve the latest data instance from the &lt;Container>.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
}

response = requests.get(url + '/myAE/myContainer/la', headers=headers)    # READ request
printResponse(response)

## Update the &lt;Container Resource

With this request we will set the *MinimumNumberOfInstances* (mni) attribute to a new value.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
}

data = '''
{
  "m2m:cnt": {
    "mni": 1000
  }
}
'''

response = requests.put(url + '/myAE/myContainer', headers=headers, data=data)    # UPDATE request
printResponse(response)

The CSE returns the updated attributes of the resource. Also note the change of the *lastModificationTime* (lt) and *status* (st) attributes.

## Check the &lt;Container> resource
Retrieve the &lt;Container> resource to see all the changes and its current state.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '0',                      # Request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
}

response = requests.get(url + '/myAE/myContainer', headers=headers)    # READ request
printResponse(response)